In [1]:
import numpy as np
import arcode
import codec
import struct
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
def ar_decode(aname, dname):
    ar = arcode.ArithmeticCode(False)
    ar.decode_file(aname, dname)

    file_bin = open(dname,'rb')
    s = file_bin.read()
    H,W,C = struct.unpack('3i',s[:12])
    #print('H= ' + str(H) + ' ; W= ' + str(W) + ' ; C= ' + str(C))
    
    bytes=[]
    for i in range(H*W*C):
        bytes.append(struct.unpack('b',s[12+i:12+i+1]))
    file_bin.close()
    bin_data = np.array(bytes).reshape(H,W,C)
    bin_data = bin_data.flatten()
    
    return bin_data

In [47]:
'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_884/COMPINT'
nx=256
ny=384
ncomp=4
'''


pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Gc_16164/COMPINT'
nx=128
ny=192
ncomp=4


'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_32324/COMPINT'
nx=64
ny=96
ncomp=4
'''

print(pref)

/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Gc_16164/COMPINT


In [48]:
i_f = 7
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 128, 192, 4) , mean = 127.73268636067708 , std = 36.7339617665188 , Max = 226.0 , Min = 48.0 , type=float64
t_C_0 shape: (1, 128, 192) , mean = 91.43367513020833 , std = 11.806028958191112 , Max = 133.0 , Min = 61.0 , type=float64
t_C_1 shape: (1, 128, 192) , mean = 94.9801025390625 , std = 13.09076523219654 , Max = 147.0 , Min = 48.0 , type=float64
t_C_2 shape: (1, 128, 192) , mean = 163.01383463541666 , std = 12.481567295196928 , Max = 226.0 , Min = 108.0 , type=float64
t_C_3 shape: (1, 128, 192) , mean = 161.50313313802084 , std = 12.467692230659456 , Max = 209.0 , Min = 101.0 , type=float64


In [49]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (128, 192) , mean = 33.503133138020836 , std = 12.467692230659456 , Max = 81 , Min = -27 , type=int8


In [50]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen24576
size124588
size216835
ratio1= 1.00048828125
ratio2= 0.685017903646


In [51]:
aname = oname + 'arith'
dname = 'dAc.dat'
dcode = ar_decode(aname, dname)

dlen = len(dcode)
print('dlen= ' + str(dlen))

mse = np.mean(np.square(dcode-data_t.flatten()))
print('mse = ' + str(mse))

dlen= 24576
mse = 0.0


In [52]:
fs = 1; fe = 25
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])
d_lens = np.ones([nf,4])
mses = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        dcode = ar_decode(aname, dname)
        dlen = len(dcode)
        print('dlen= ' + str(dlen))

        idx = i-fs
        
        if dlen == olen:
            d_lens[idx,idc] = 0
            mse = np.mean(np.square(dcode-data_t.flatten()))
            mses[idx,idc] = mse
            print('mse = ' + str(mse))
        if dlen != olen:
            d_lens[idx,idc] = 1
            mses[idx,idc] = 99.99
            print('!Length differ!')
        
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
        print('')
    
    print('\n')

1
  t_C shape: (1, 128, 192, 4) , mean = 127.77382405598958 , std = 36.6595091156784 , Max = 230.0 , Min = 41.0 , type=float64
 channel0 shape: (128, 192) , mean = -35.367757161458336 , std = 14.59879603994896 , Max = 2 , Min = -74 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17549 Bytes
ratio1= 1.00048828125
ratio2= 0.714070638021
dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = -31.765787760416668 , std = 15.194030277880938 , Max = 26 , Min = -87 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17971 Bytes
ratio1= 1.00048828125
ratio2= 0.731241861979
dlen= 24576
mse = 0.0

 channel2 shape: (128, 192) , mean = 34.129475911458336 , std = 15.834175391769234 , Max = 102 , Min = -26 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17951 Bytes
ratio1= 1.00048828125
ratio2= 0.730428059896
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = 32.099365234375 , std = 15.034781384834996 , Max = 85 , Min = -20 , type=int8
   o

dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = -34.838094075520836 , std = 17.212501674087722 , Max = 35 , Min = -104 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18746 Bytes
ratio1= 1.00048828125
ratio2= 0.762776692708
dlen= 24576
mse = 0.0

 channel2 shape: (128, 192) , mean = 37.125651041666664 , std = 18.88423950095078 , Max = 113 , Min = -23 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 19134 Bytes
ratio1= 1.00048828125
ratio2= 0.778564453125
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = 35.339396158854164 , std = 16.842894923409123 , Max = 100 , Min = -13 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18598 Bytes
ratio1= 1.00048828125
ratio2= 0.756754557292
dlen= 24576
mse = 0.0



9
  t_C shape: (1, 128, 192, 4) , mean = 127.68467203776042 , std = 43.334764208189185 , Max = 216.0 , Min = 33.0 , type=float64
 channel0 shape: (128, 192) , mean = -44.331095377604164 , std = 10.75629069837313 , Max =

   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17832 Bytes
ratio1= 1.00048828125
ratio2= 0.7255859375
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = 28.098347981770832 , std = 24.777001275229228 , Max = 95 , Min = -27 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17451 Bytes
ratio1= 1.00048828125
ratio2= 0.710083007812
dlen= 24576
mse = 0.0



16
  t_C shape: (1, 128, 192, 4) , mean = 127.85293579101562 , std = 36.004273463701985 , Max = 210.0 , Min = 55.0 , type=float64
 channel0 shape: (128, 192) , mean = -35.528889973958336 , std = 12.966838287998852 , Max = -2 , Min = -73 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17248 Bytes
ratio1= 1.00048828125
ratio2= 0.701822916667
dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = -32.214599609375 , std = 12.534058746004069 , Max = 5 , Min = -72 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17219 Bytes
ratio1= 1.00048828125
ratio2= 0.700642903646
dlen= 245

dlen= 24576
mse = 0.0



23
  t_C shape: (1, 128, 192, 4) , mean = 127.74155680338542 , std = 36.05922207514889 , Max = 216.0 , Min = 43.0 , type=float64
 channel0 shape: (128, 192) , mean = -34.390218098958336 , std = 16.170682082148296 , Max = 4 , Min = -79 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18476 Bytes
ratio1= 1.00048828125
ratio2= 0.751790364583
dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = -31.082438151041668 , std = 15.062155538703157 , Max = 37 , Min = -85 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18208 Bytes
ratio1= 1.00048828125
ratio2= 0.740885416667
dlen= 24576
mse = 0.0

 channel2 shape: (128, 192) , mean = 32.917399088541664 , std = 15.974156578408438 , Max = 88 , Min = -10 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18438 Bytes
ratio1= 1.00048828125
ratio2= 0.750244140625
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = 31.521484375 , std = 15.226286122120277 , Max = 85 , Mi

In [53]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.00048828125
   ratio_2= 0.7251180013020834
 
2
   ratio_1= 1.00048828125
   ratio_2= 0.6006673177083334
 
3
   ratio_1= 1.00048828125
   ratio_2= 0.6989644368489584
 
4
   ratio_1= 1.00048828125
   ratio_2= 0.7343648274739584
 
5
   ratio_1= 1.00048828125
   ratio_2= 0.750762939453125
 
6
   ratio_1= 1.00048828125
   ratio_2= 0.728302001953125
 
7
   ratio_1= 1.00048828125
   ratio_2= 0.688018798828125
 
8
   ratio_1= 1.00048828125
   ratio_2= 0.7633463541666666
 
9
   ratio_1= 1.00048828125
   ratio_2= 0.6644083658854166
 
10
   ratio_1= 1.00048828125
   ratio_2= 0.6862691243489584
 
11
   ratio_1= 1.00048828125
   ratio_2= 0.6823018391927084
 
12
   ratio_1= 1.00048828125
   ratio_2= 0.630645751953125
 
13
   ratio_1= 1.00048828125
   ratio_2= 0.7376505533854166
 
14
   ratio_1= 1.00048828125
   ratio_2= 0.747283935546875
 
15
   ratio_1= 1.00048828125
   ratio_2= 0.7172342936197916
 
16
   ratio_1= 1.00048828125
   ratio_2= 0.6980692545572916
 
17
   ratio_1= 1.00048